In [2]:
%load_ext zipline

In [149]:
%%zipline --start 2017-1-1 --end 2018-1-1  --capital-base 10000 -o ../data/ensemble1.pickle

'''
Ensemble Strategy on a combination of different Assets.
Pipelines are used to filter out assets that has bad indicators.
'''
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import math
from zipline.finance import commission, slippage  
from zipline.api import (
    sid, 
    symbol,
    schedule_function, 
    date_rules, 
    time_rules,
    order_target_percent,
    record) 

import talib
from talib import EMA, BBANDS, RSI, ATR
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from zipline.api import attach_pipeline, pipeline_output
from zipline.pipeline.factors import (
    ExponentialWeightedMovingAverage, 
    AverageDollarVolume, 
    RSI, # weak indicator
    BollingerBands,
    #AnnualizedVolatility,
    #RollingLinearRegression,
    #RollingPearson, 
    MaxDrawdown,
    Returns,
    VWAP,
    TrueRange
)
    

def initialize(context):
    
    context.sectors = dict()
    context.sectors['Technology'] = [
        symbol('AAPL'),
        #symbol('GOOGL'),
        #symbol('NVDA'),
        #symbol('FB'), 
        #symbol('MSFT'),
    ]
    
    context.sectors['Financial'] = [
        symbol('JPM'),
        #symbol('BRK-A'),
        #symbol('BAC')
    ] 
    
    context.sectors['Healthcare'] =[
        symbol('JNJ'),
        #symbol('PFE'), 
        #symbol('UNH')
    ]    
    
    context.sectors['Service'] =[
        symbol('f'),  
        #symbol('WMT'), 
        symbol('NFLX')
    ]
    
    context.sectors['Utility'] =[
        symbol('NEE'),  
        #symbol('DUK'), 
        #symbol('SRE')
    ]
    
    context.sectors['BasicMaterial'] =[
        symbol('CVX'),
        #symbol('PSX'),
        #symbol('CEO') 
    ]
    
    context.sectors['IndustrialGoods'] =[
        symbol('BA'),
        #symbol('MMM'),
        #symbol('HON') 
    ]
    
    context.sectors['ConsumerGoods'] =[
        symbol('KO'),
        #symbol('PM'),
        #symbol('NKE')
    ]
    
    
    context.assets = \
    context.sectors['Technology'] + \
    context.sectors['Financial'] + \
    context.sectors['Healthcare'] + \
    context.sectors['Service'] + \
    context.sectors['Utility'] + \
    context.sectors['BasicMaterial'] + \
    context.sectors['IndustrialGoods'] + \
    context.sectors['ConsumerGoods']  
    context.assets = [symbol('QQQ')]
    
    
    context.pct_per_asset = 1.0 / len(context.assets)  
    strategies = [
        getMaxDrawDownIndicator, #0
        getRsiIndicator,         #1
        getbbandIndicator,   #2
        getAtrIndicator,  #3
        getEMAIndicator,       #4
        getMomentumIndicator,  #5
        getVwmaIndicator  #6
    ]
    context.strategies = strategies[6:]
    #context.strategies = strategies
    context.Hparameters = {
        'long_win':30, 
        'short_win':15, 
        'span':15,               #for exponential moving average factor
        'momentumDelta': 0.02,   #for momentum factor
        'delta': 0.005,
        'high_rsi': 80,  #for RSI factor
        'low_rsi': 30}   #for RSI factor
    context.pipe = get_pipeline(context.Hparameters)
    schedule_function(
        rebalance, 
        date_rules.every_day(), 
        time_rules.market_open()
    ) 

    
def get_pipeline(Hparameters):
    pipe = Pipeline()  
    # Construct Factors.
    short_win = Hparameters['short_win']
    long_win = Hparameters['long_win']
    #delta = Hparameters['delta']
    
    ### exponential moving average factor:
    ema_short = ExponentialWeightedMovingAverage(
        inputs=[USEquityPricing.close], 
        window_length=short_win,
        decay_rate=(1 - (2.0 / (1 + Hparameters['span']))))
    ema_long = ExponentialWeightedMovingAverage(
        inputs=[USEquityPricing.close], 
        window_length=long_win,
        decay_rate=(1 - (2.0 / (1 + Hparameters['span']))))
    
    ema_ratio = ema_short / ema_long
    
    ### volume weighted moving average factor:
    vwma_short = VWAP(
        inputs=[USEquityPricing.close,
                USEquityPricing.volume], 
        window_length=short_win)
    
    vwma_long = VWAP(
        inputs=[USEquityPricing.close,
                USEquityPricing.volume],  
        window_length=long_win)
     
    vwma_ratio = vwma_short / vwma_long
    
    ### BBANDs factor
    bbands = BollingerBands(
        inputs=[USEquityPricing.close],
        window_length=long_win,
        k=2)
   
    ### MAXDrawDown factor
    maxdrawdown = MaxDrawdown(
        inputs=[USEquityPricing.close],
        window_length=long_win)

    ### RSI factor
    rsi = RSI(
        inputs=[USEquityPricing.close],
        window_length=short_win)
    
 
    ### True range factor
    atr = TrueRange(
        inputs=[USEquityPricing.high,
                USEquityPricing.low,
                USEquityPricing.close],
        window_length=2) 
 
 

    
    #ema_rank = ema_ratio.rank(mask=ema_short > 10.0)
    
    #longs = ema_rank.top(500) & (ema_ratio > 1 + delta)  
    #shorts = ema_rank.bottom(500) & (ema_ratio < 1 - delta)
    #longs =   ema_ratio > 1 + delta   
    #shorts =  ema_ratio < 1 - delta 
     
    # Register outputs. 
    pipe_columns = { 
        'ema_short':ema_short,
        'ema_long':ema_long,
        'ema_ratio':ema_ratio,
        #'longs':longs,
        #'shorts':shorts,
        'bbands':bbands,
        'maxdrawdown': maxdrawdown,  
        'rsi': rsi,
        #'atr': atr,
        'vwma_short':vwma_short,
        'vwma_long':vwma_long,
        'vwma_ratio':vwma_ratio
    }
    
    # Construct a filter to narrow the universe. ignore stocks lower than 5 dollars
    ema_filter = (ema_short > 5 )  
    # Remove rows for which the Filter returns False. 
    pipe = Pipeline(columns = pipe_columns,screen = ema_filter)
    attach_pipeline(pipe, 'my_pipeline')  
    return pipe
    
    
def before_trading_start(context, data):
    # Access results using the name passed to `attach_pipeline`.
    results = pipeline_output('my_pipeline')
    
    #context.long_assets = results.longs.index
    #context.short_assets = results.shorts.index
    
    # Store pipeline results for use by the rest of the algorithm.
    context.results = results
    #print results 
  

def getVwmaIndicator(context, asset, data):
    if asset not in context.results['vwma_ratio']:
        return 0
    delta = context.Hparameters['delta']
    vwma_ratio = context.results['vwma_ratio'][asset]
    if vwma_ratio > 1 + delta:
        return 1
    if vwma_ratio < 1 - delta:
        return -1
    return 0


def getRsiIndicator(context, asset, data):
    if asset not in context.results['rsi']:
        return 0
    rsi = context.results['rsi'][asset]
    if rsi > context.Hparameters['high_rsi']:
        return -1
    if rsi < context.Hparameters['low_rsi']:
        return 1
    return 0 

     
def getAtrIndicator(context, asset, data):
    '''Average True Range'''
    if not asset: asset = context.asset
    price = data.current(asset, 'price')
    days = context.Hparameters['short_win']
    history = data.history(asset,
                        fields=['high', 'low', 'close'],
                        bar_count=days,
                        frequency='1d')

    # Calculate the ATR for the asset
    atr = talib.ATR(history['high'],
                    history['low'],
                    history['close'],
                    timeperiod=days-1)[-1]

    # Use the close price from yesterday because we trade at market open
    prev_close = history['close'][-2]
    indicator = 0
    # An upside breakout occurs when the price goes 1 ATR above the previous close
    if price > prev_close + atr:
        indicator = 1

    # A downside breakout occurs when the previous close is 1 ATR above the price
    if price < prev_close - atr:
        indicator = -1

    return indicator 

    
def UnusedGetAtrIndicator(context, asset, data):
    if asset not in context.results['atr']:
        return 0
    delta = context.Hparameters['delta'] 
    atr = context.results['atr'][asset]
    price = data.current(asset, 'price') 
    history = data.history(asset,  fields=['close'], bar_count=2, frequency='1d')
    pre_close = history['close'][-2]
    if price > (pre_close + atr) * (1 + delta):
        return 1
    if price < (pre_close - atr) * (1 - delta):
        return -1
    return 0


def getMaxDrawDownIndicator(context, asset, data):
    if asset not in context.results['maxdrawdown']:
        return 0
    maxdrawdown = context.results['maxdrawdown'][asset]
    #print(maxdrawdown)
    if maxdrawdown < -0.2:
        return -1
    if maxdrawdown > -0.2:
        return 1
    return 0
                  
def getbbandIndicator(context, asset, data):
    if asset not in context.results['bbands']:
        return 0
    low, middle, high = tuple(context.results['bbands'][asset])
    price = data.current(asset, 'price') 
    indicator = 0
    if price < low: 
        indicator = 1
    if price > high:
        indicator = -1
    return indicator
        
def getMomentumIndicator(context, asset, data): 
    price = data.current(asset, 'price') 
    delta = context.Hparameters['momentumDelta']
    #delta = 0.02
    if asset not in context.results['ema_short']:
        return 0
    avg = context.results['ema_short'][asset] 
    if price > (1 + delta) * avg:
        return 1
    if price < (1 - delta) * avg:
        return -1
    return 0
    
def getEMAIndicator(context, asset, data):
    if asset not in context.results['ema_ratio']:
        return 0 
    delta = context.Hparameters['delta']
    ratio = context.results['ema_ratio'][asset] 
    if ratio > 1 + delta:
        return 1
    if ratio < 1 - delta:
        return -1 
    return 0
    
    
def getEnsembleIndicator(context, asset, data):  
    indicators = []
    for strategy in context.strategies: 
        indicators.append(strategy(context, asset, data))
    indicator = float(sum(indicators)) / len(indicators) 
    return indicator
    
 
    
def rebalance(context, data): 
    for i, asset in enumerate(context.assets): 
        position = context.portfolio.positions[asset].amount
        price = data.current(asset, 'price')   
        if data.can_trade(asset): 
            indicator = getEnsembleIndicator(context, asset, data) 
            if indicator > 0 and position <= 0: 
                order_target_percent(asset, context.pct_per_asset)
            if indicator < 0 and position > 0:
                order_target_percent(asset, 0)   
            record(asset, price) 
 
    
def analyze(context, perf):
    assets = context.assets
    nAssets = len(assets)
    ax1s = [None for i in range(nAssets)] 
    assets2index = dict([(asset, i) for i, asset in enumerate(assets)])  
    
    fontsize = 16  
    
    for i, asset in enumerate(assets):
        
        price = perf[asset]  
        #ax1s[i] = fig.add_subplot(fig_pos)  
        fig, fig_pos = plt.figure(i+1), '111' 
        ax1s[i] = fig.add_subplot(fig_pos) 
        color = np.random.uniform(0,1, size = 3) 
        price.plot(ax=ax1s[i], c=color) 
        ax1s[i].set_ylabel('Asset={}'.format(str(asset)[8:]))
        
     
     
    perf_trans = perf.ix[[t != [] for t in perf.transactions]]  
    buys = perf_trans.ix[
        [t[0]['amount'] > 0 for t in perf_trans.transactions]]
    sells = perf_trans.ix[
        [t[0]['amount'] < 0 for t in perf_trans.transactions]] 
    
    buy_assets = [trans[0]['sid'] for trans in buys['transactions']]  
    buy_prices = [perf[buy_assets[i]][t] for i, t in enumerate(buys.index)]     
    sell_assets = [trans[0]['sid'] for trans in sells['transactions']]  
    sell_prices = [perf[sell_assets[i]][t] for i, t in enumerate(sells.index)] 
    
    for i, buy in enumerate(buys['transactions']):
        asset = buy_assets[i]
        price = buy_prices[i]
        #print(buys.index[i], price, str(asset)[8:])
        ax = ax1s[assets2index[asset]]
        ax.plot(buys.index[i], price, '^', markersize=10, color='m')  
        #plt.gcf().set_size_inches(15, 13)
        
    for i, sell in enumerate(sells['transactions']):
        asset = sell_assets[i]
        price = sell_prices[i]
        ax = ax1s[assets2index[asset]] 
        ax.plot(sells.index[i], price, 'v', markersize=10, color='k') 
        #plt.gcf().set_size_inches(15, 13)
    
    ### print portfolio profit
    portfolio = perf.portfolio_value #- perf.portfolio_value.iloc[0] 
    fig, fig_pos = plt.figure(nAssets+1), '111'  
    ax2 = fig.add_subplot(fig_pos)
    portfolio.plot(ax=ax2)
    ax2.set_ylabel('portfolio value in $')
    #plt.gcf().set_size_inches(15, 13)
    plt.show()

 


SymbolNotFound: Symbol 'QQQ' was not found.

In [148]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
perf = pd.read_pickle('../data/ensemble1.pickle')
perf.columns
print("Benchmark Profit:{:>16.5f} \nOur strategy Profit: {:>12.5f}".format(perf['benchmark_period_return'][-1], perf['algorithm_period_return'][-1]))
#perf[['benchmark_period_return','algorithm_period_return']]


def analyze(perf):
    #assets = context.assets
    assets = perf.columns[:10] 
    #assets = context.assets 
    nAssets = len(assets)
    ax1s = [None for i in range(nAssets)] 
    assets2index = dict([(asset, i) for i, asset in enumerate(assets)])  
    #fig = plt.figure()
    fontsize = 16  
    
    for i, asset in enumerate(assets):
        fig, fig_pos = plt.figure(i+1), '111'  
        ax1s[i] = fig.add_subplot(fig_pos) 
        price = perf[asset] 
        color = np.random.uniform(0,1, size = 3)
        #print(color)
        price.plot(ax=ax1s[i], c=color) 
        ax1s[i].set_ylabel('Asset={}'.format(str(asset)[8:]))
     
     
    perf_trans = perf.ix[[t != [] for t in perf.transactions]]  
    buys = perf_trans.ix[
        [t[0]['amount'] > 0 for t in perf_trans.transactions]]
    sells = perf_trans.ix[
        [t[0]['amount'] < 0 for t in perf_trans.transactions]] 
    
    buy_assets = [trans[0]['sid'] for trans in buys['transactions']]  
    buy_prices = [perf[buy_assets[i]][t] for i, t in enumerate(buys.index)]     
    sell_assets = [trans[0]['sid'] for trans in sells['transactions']]  
    sell_prices = [perf[sell_assets[i]][t] for i, t in enumerate(sells.index)] 
    
    for i, buy in enumerate(buys['transactions']):
        asset = buy_assets[i]
        price = buy_prices[i]
        #print(buys.index[i], price, str(asset)[8:])
        ax = ax1s[assets2index[asset]]
        ax.plot(buys.index[i], price, '^', markersize=10, color='m')  
    for i, sell in enumerate(sells['transactions']):
        asset = sell_assets[i]
        price = sell_prices[i]
        ax = ax1s[assets2index[asset]] 
        ax.plot(sells.index[i], price, 'v', markersize=10, color='k') 
        
 
    fig, fig_pos = plt.figure(nAssets+1), '111'  
    ax2 = fig.add_subplot(fig_pos)
    portfolio = perf.portfolio_value #- perf.portfolio_value.iloc[0] 
    portfolio.plot(ax=ax2)
    ax2.set_ylabel('portfolio value in $')
    plt.gcf().set_size_inches(22, 18)
    plt.show()
 
    
#analyze(perf)

Benchmark Profit:         0.37983 
Our strategy Profit:      0.38027
